### NATURAL JOIN

Pissa, the ever-expanding pizza delivery enterprise, has a new challenge for you. They're interested in discovering which type of pizza generates the most revenue.

Identify the top-selling pizza category using your knowledge on NATURAL JOIN.

Here is the pizza schema for your reference:

Schema Diagram [skipped]

### Instructions
    - Calculate total_revenue based on SUM of price * quantity; taking price from p (pizzas) and quantity from od (order_details) table.
    - NATURAL JOIN the pizzas and pizza_type tables.
    - GROUP the records by category from pt (pizza_type) table.
    - ORDER the details by total_revenue in descending order and LIMIT to 1 to fetch only the top revenue pizza.

In [ ]:
SELECT 
    pt.category,
	-- Calculate total_revenue 
    sum(p.price*od.quantity) as total_revenue
FROM order_details AS od
-- NATURAL JOIN all tables
NATURAL JOIN pizzas AS p 
NATURAL JOIN pizza_type AS pt
-- GROUP the records by category from pizza_type table
GROUP BY pt.category
-- ORDER by total_revenue and limit the records
ORDER BY total_revenue desc
LIMIT 1

### The world of JOINS

As a consulting data engineer, Pissa now wants your advanced Snowflake SQL skills to get some insights in their sales and revenue.

Apply your knowledge on Joins to get the desired result.

### Instructions 1/3
    - Ensure that all orders from the orders table are included in the result, regardless of whether they have corresponding entries in the order_details table.

In [ ]:
SELECT COUNT(o.order_id) AS total_orders
FROM orders AS o
-- Use appropriate JOIN
LEFT JOIN order_details AS od
ON o.order_id = od.order_id

### Instructions 2/3
    - Calculate the total revenue using price column from pizzas table and quantity column of order_details table respectively.
    - Use appropriate JOIN to include all records from the pizzas table.

In [ ]:
SELECT COUNT(o.order_id) AS total_orders,
        AVG(p.price) AS average_price,
        -- Calculate total revenue
        SUM(p.price * od.quantity) AS total_revenue	
FROM orders AS o
-- Use appropriate JOIN
LEFT JOIN order_details AS od
ON o.order_id = od.order_id
-- Use appropriate JOIN with pizzas table
RIGHT JOIN pizzas AS p
ON od.pizza_id = p.pizza_id


### Instructions 3/3
    - Select pizza name from pizza_type by performing a NATURAL JOIN with the pizza_type table.

In [ ]:
SELECT COUNT(o.order_id) AS total_orders,
        AVG(p.price) AS average_price,
        -- Calculate total revenue
        SUM(p.price * od.quantity) AS total_revenue,
        -- Get the name from pizza_type table
		pt.name AS pizza_name
FROM orders AS o
-- Use appropriate JOIN
LEFT JOIN order_details AS od
ON o.order_id = od.order_id
-- Use appropriate JOIN with pizzas table
RIGHT JOIN pizzas p
ON od.pizza_id = p.pizza_id
-- NATURAL JOIN pizza_type table
NATURAL JOIN pizza_type pt
GROUP BY pt.name, pt.category
ORDER BY total_revenue desc, total_orders desc

### LATERAL JOIN

Pissa, the expanding pizza delivery enterprise, is now taking your expertise on identifying some trends.

They want to find out which dates feature maximum sales for each type of pizza. This information will enable them to pinpoint trends and optimize future promotions.

### Instructions
    - Complete the query to find the max_quantity from the LATERAL query.
    - Join the appropriate tables.
    - Use LATERAL join to calculate max_quantity and complete the subquery to find max of quantity.
    - Filter the subquery on pizza_type_id referencing outer pizza_type table.

In [ ]:
SELECT pt.name, 
	   pt.category, 
       o.order_date,
       -- Get max quantity from lateral query
       x.max_quantity
FROM pizzas AS pz
JOIN pizza_type AS pt ON pz.pizza_type_id = pt.pizza_type_id
JOIN order_details AS od ON pz.pizza_id = od.pizza_id
-- Join with orders table
JOIN orders AS o ON od.order_id = o.order_id,    
LATERAL (
    -- Select max of order_details quantity
    SELECT MAX(od2.quantity) AS max_quantity
    FROM order_details AS od2
    -- Join with pizzas table
    JOIN pizzas AS pz2 
        ON od2.pizza_id = pz2.pizza_id
    -- Filtering condition for the subquery
    WHERE pz2.pizza_type_id = pz.pizza_type_id
) AS x
WHERE od.quantity = x.max_quantity
GROUP BY pt.name, pt.category, o.order_date, x.max_quantity
ORDER BY pt.name;

### Subqueries

Pissa, the expanding pizza delivery enterprise, is now taking your expertise on identifying some trends.

They want to streamline its pizza offerings by identifying underperforming pizza types. Your task is to find the pizza types that are ordered less frequently than the average for all types.

### Instructions
    - Complete the JOIN clauses to join the pizzas, and order_details tables appropriately.
    - Use the GROUP BY clause to group the results by name and category from the pizza_type table.
    - Fill in the subquery to find the AVG of total_quantity.
    - Order the results by total_quantity in ascending order.

In [ ]:
SELECT 
    pt.name, 
    pt.category, 
    SUM(od.quantity) AS total_quantity
FROM pizza_type AS pt
-- Join pizzas and order_details table
join pizzas AS pz
    ON pt.pizza_type_id = pz.pizza_type_id
join order_details AS od
    ON pz.pizza_id = od.pizza_id
-- Group by name and category
group by pt.name, pt.category
HAVING SUM(od.quantity) < (
    -- Calculate AVG of total_quantity 
    SELECT avg(total_quantity)
    FROM (
        SELECT SUM(od2.quantity) AS total_quantity
        FROM pizzas AS pz2
        JOIN order_details AS od2
            ON pz2.pizza_id = od2.pizza_id
        GROUP BY pz2.pizza_type_id
    ) AS sub
)
-- Order  by total_quantity in ascending order
order by total_quantity

### CTEs

Pissa, the company you're consulting for, is planning a promotional campaign and needs your expertise.

The campaign aims to spotlight their most popular pizza based on total orders.

Additionally, they're thinking about introducing a value meal featuring their least expensive pizza.

Your task as a consulting data engineer is to identify both these pizzas.

### Instructions
    - Create a CTE named most_ordered and limit the results to one.
    - Create another CTE, called cheapest_pizza and filter for the cheapest pizza using a subquery to find the minimum price.
    - Complete the query to select pizza_id and total_qty aliased as metric from most_ordered CTE.
    - Include pizza_id and price aliased as metric from cheapest_pizza CTE. Note, maintain order of the columns.

In [ ]:
-- Create a CTE named most_ordered and limit the results 
with most_ordered as (
    SELECT pizza_id, SUM(quantity) AS total_qty 
    FROM order_details GROUP BY pizza_id ORDER BY total_qty DESC
    limit 1
)
-- Create CTE cheapest_pizza where price is equals to min price from pizzas table
, cheapest_pizza as (
    SELECT pizza_id, price
    FROM pizzas 
    WHERE price = (SELECT min(price) FROM pizzas)
    LIMIT 1
)
-- Select pizza_id and total_qty aliased as metric from first cte most_ordered
SELECT pizza_id, 'Most Ordered' AS Description, total_qty as metric
FROM most_ordered
UNION ALL
-- Select pizza_id and price aliased as metric from second cte cheapest_pizza
SELECT pizza_id, 'Cheapest' AS Description, price as metric
FROM cheapest_pizza

### Early filtering

Pissa has now asked for your expertise to optimize the performance of their database queries. They suspect that their existing query is not efficient enough and take too long to run.

The goal is to retrieve the orders made after November 01, 2015, and only the pizzas in the 'Veggie' category.

Complete the given SQL query by implementing early filtering techniques.

### Instructions
    - Use the filtered_orders CTE to select order_id and order_date from orders table, filtering to only include orders made after November 1, 2015.
    - Use the filtered_pizza_type CTE to select the name and pizza_type_id from the pizza_type table, filtering to only include pizzas in the 'Veggie' category.
    - Retrieve the records from the filtered_orders CTE.
    - Join the filtered_pizza_type CTE based on the pizza_type_id column using ON clause.

In [ ]:
WITH filtered_orders AS (
  -- Select order_id, order_date and filter records where order_date is greater than November 1, 2015.
  SELECT order_id, order_date 
  FROM orders 
  WHERE order_date > '2015-11-01'
)

, filtered_pizza_type AS (
  -- Select name, pizza_type_id and filter the pizzas which has Veggie category
  SELECT name, pizza_type_id 
  FROM pizza_type 
  WHERE category = 'Veggie'
)

SELECT o.order_id, o.order_date, pt.name, od.quantity
-- Get the details from filtered_orders CTE
FROM filtered_orders AS o
JOIN order_details AS od ON o.order_id = od.order_id
JOIN pizzas AS p ON od.pizza_id = p.pizza_id
-- JOIN CTE filtered_pizza_type on common column
JOIN filtered_pizza_type as pt ON pt.pizza_type_id = p.pizza_type_id

### Querying JSON data

Yelpto, a leading platform for discovering local businesses, seeks your expertise as a consulting data engineer.

They aim to explore the restaurant industry, focusing on Italian cuisine that's wheelchair accessible and open on weekends.

You'll work with the yelp_business_data table, particularly the name, categories, attributes, and hours columns.

Here is the schema of the yelp_business_data table:

<code>
CREATE TABLE yelp_business_data (
  business_id VARCHAR(50),
  name VARCHAR(100),
  ...
  attributes VARIANT,
  categories VARCHAR(200),
  hours VARIANT
)
</code>

Note, both attributes and hours are VARIANT types.

### Instructions 1/2
    - Extract the WheelchairAccessible field from the attributes column and cast it to STRING using :: notation.
    - Extract Saturday, Sunday from hours column and cast it to STRING using :: notation.

In [ ]:
SELECT
  name,
  categories,
  -- Select WheelchairAccessible from attributes converting it to STRING
  attributes:WheelchairAccessible::string AS wheelchair_accessible,
  -- Select Saturday, Sunday from hours converting it to STRING
  (hours:Saturday::string IS NOT NULL OR hours:Sunday::string IS NOT NULL) AS open_on_weekend
FROM
  yelp_business_data

### Instructions 2/2
    - Filter the query results where wheelchair_accessible is 'True' and open_on_weekend is 'true'.
    - Filter the query for categories that have 'Italian' in them.

In [ ]:
SELECT
  name,
  categories,
  -- Select WheelchairAccessible from attributes converting it to STRING
  attributes:WheelchairAccessible::STRING AS wheelchair_accessible,
  -- Select Saturday, Sunday from hours converting it to STRING
  (hours:Saturday::STRING IS NOT NULL OR hours:Sunday::STRING IS NOT NULL) AS open_on_weekend
FROM
  yelp_business_data
WHERE
	-- Filter where wheelchair_accessible is 'True' and open_on_weekend is 'true'
	wheelchair_accessible = 'True' and open_on_weekend = 'true'
    -- Filter further where categories is having Italian in it
    and categories like '%Italian%'

    

### JSONified

As you continue working as a consulting data engineer for Yelpto, they are looking for more insights from you.

This time, they want to find out which restaurants allow dogs and are touristy.

The catch? Some of the attributes are deeply nested within a JSON string.

Leverage your acquired skills to help them get those insights in the upcoming steps.

### Instructions 1/2
    - Create a CTE named dogs_allowed.
    - Filter businesses where the DogsAllowed attribute in the attributes column, when cast to STRING, is True.

In [ ]:
-- Create CTE dogs_allowed.
WITH dogs_allowed AS (
  SELECT * 
  FROM yelp_business_data
  WHERE attributes:DogsAllowed::STRING  NOT ILIKE '%None%'
  -- Filter data where DogsAllowed is True.
  AND attributes:DogsAllowed::STRING = 'True' 
)

SELECT
	business_id, name
FROM dogs_allowed

### Instructions 2/2
    - From second CTE tourist_places, convert the Ambience attribute in the attributes columns into valid JSON using PARSE_JSON.
    - From the valid parsed JSON, fetch the touristy attribute and check if it is true when cast to BOOLEAN.

In [ ]:
WITH dogs_allowed AS (
  SELECT * 
  FROM yelp_business_data
  WHERE attributes:DogsAllowed::STRING  NOT ILIKE '%None%'
  AND attributes:DogsAllowed::STRING = 'True' 
)

, touristy_places AS (
  SELECT *
  FROM yelp_business_data
  WHERE attributes:Ambience NOT ILIKE '%None%'
    AND attributes:Ambience IS NOT NULL
    AND attributes:Ambience NOT ILIKE '%u\'%'
    -- Convert Ambience attribute in the attributes columns into valid JSON using PARSE_JSON.
    -- From Valid JSON, fetch the touristy attribute and check if it is true when casted to BOOLEAN.
    AND PARSE_JSON(attributes:Ambience):touristy::BOOLEAN = true
)

SELECT
	d.business_id,
    d.name
FROM dogs_allowed d
JOIN touristy_places t
	ON d.business_id = t.business_id